<a href="https://colab.research.google.com/github/grace3999/USV_Python/blob/colab/Notebooks/2_xr_Annotations_from_netcdf_8features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#mount google drive containings required files: 1) csv of annotation features, 2) netcdf files
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip install netcdf4

    100% |████████████████████████████████| 3.9MB 7.4MB/s 
    100% |████████████████████████████████| 307kB 21.6MB/s 


In [0]:
import numpy as np
import pandas as pd
import os
import re
from scipy import stats
import xarray as xr

#visualizing results
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [0]:
#may need to be updated based on file naming scheme
def get_file_info(path, order):
    """takes in a file path for annotation selections table and finds the animal_number and session and saves each accordingly. 
    each file should be named with animal number and exp (e.g. 100_CPA.Table.1.selections)"""
    
    if order == 'animal':
      animal_number, session = re.split('_|-', path.split('/')[-1].split('.')[0])[0:2]
    else:
      session, animal_number = re.split('_|-', path.split('/')[-1].split('.')[0])[0:2] 
    
    print(animal_number, session)
    
    return animal_number, session

In [0]:
#need to be updated with animal naming scheme (e.g. int(animal_number) vs animal_number)
def create_annot_dataset(path, animal_number, session, annot_data, size_random):
    """takes in path of netcdf file (created using notebook 1) and corresponding annotations (created using notebook 0)
    and creates a new data set of only slices correspondingn to annotations (and random noise if set to True)"""
    
    #open xarray Dataset
    data = xr.open_dataset(path)
    print(data['slices'].shape)
    
    #get time_stamps of animal's annotations, select slices and save
    yes = annot_data[(annot_data['animal_number'] ==  int(animal_number)) & (annot_data['session'] ==  session)]['time_stamp'].sort_values().values
    data_yes = data.sel(slices = yes)
    
    #get slice indexs (e.g. time_stamps) that are not annotations, select slices and save
    slice_indexes = data['slices'].values
    no = np.setdiff1d(slice_indexes,yes)
    print(yes.shape[0] + no.shape[0])
    no_short = np.random.choice(no, size_random, replace=False)
    data_no = data.sel(slices = no_short)
    
    return data_yes, data_no

In [0]:
def compute_spectral_features(Dataset):
    """takes in netcdf dataset and computes 8 features"""
    
    spec_power = []
    spec_purs = []
    spec_centroid = []
    spec_spread = []
    spec_skewness = []
    spec_kurtosis = []
    spec_slope = []
    spec_roll_off = []
    
    freq_array = Dataset['freq'].values
    
    #compute power sum using groupby
    spec_power = Dataset.groupby('slices').sum(xr.ALL_DIMS)['__xarray_dataarray_variable__'].values
    
    #compute other features for each slice individually
    for value in Dataset['slices'].values:
        
        spec_pur = stats.gmean(Dataset.sel(slices = value)['__xarray_dataarray_variable__'].values, axis = None) / Dataset.sel(slices = value)['__xarray_dataarray_variable__'].values.mean()
        
        mag_array = Dataset['__xarray_dataarray_variable__'].sel(slices=value).sum(dim = 'times').values
        mag_probs = mag_array/sum(mag_array)
        freq_mag = freq_array*mag_probs
        
        spec_cent = sum(freq_mag)
        spec_spr = np.var(freq_mag)
        spec_skew = stats.skew(freq_mag)
        spec_kurt = stats.kurtosis(freq_mag)
        slope, intercept, r_value, p_value, std_err = stats.linregress(freq_array, freq_mag)
        spec_ro = .95*sum(freq_mag)
        
        spec_purs.append(spec_pur)
        spec_centroid.append(spec_cent)
        spec_spread.append(spec_spr)
        spec_skewness.append(spec_skew)
        spec_kurtosis.append(spec_kurt)
        spec_slope.append(slope)
        spec_roll_off.append(spec_ro)
        
    return spec_power, spec_purs, spec_centroid, spec_spread, spec_skewness, spec_kurtosis, spec_slope, spec_roll_off

In [0]:
#need to be updated with animal naming scheme (e.g. int(animal_number) vs animal_number)
def create_annotation_slice_features(path, order, annot_data, size_random):
    """takes in path of netcdf file (created using notebook 1), corresponding annotations (created using notebook 0),
    and size desired for random slicecs from each file.
    uses create_annot_dataset and compute_spectral_features functions to create a new data set of computed features
    for only slices correspondingn to annotations and random noise"""
    
    
    animal_number, session = get_file_info(path, order)

    print(animal_number, session)
    
    #create datasets of slices of known annotations and a random selection of noise
    data_yes, data_no = create_annot_dataset(path, animal_number, session, annot_data, size_random)
    
    #compute spectral features
    yes_spec_power, yes_spec_purs, yes_spec_centroid, yes_spec_spread, yes_spec_skewness, yes_spec_kurtosis, yes_spec_slope, yes_spec_roll_off = compute_spectral_features(data_yes)
    no_spec_power, no_spec_purs, no_spec_centroid, no_spec_spread, no_spec_skewness, no_spec_kurtosis, no_spec_slope, no_spec_roll_off = compute_spectral_features(data_no)

    #add computed features to exisiting dataframe of known annotations
    annot_yes = annot_data[(annot_data['animal_number'] ==  int(animal_number)) & (annot_data['session'] ==  session)].sort_values(by=['time_stamp'])
    annot_yes['power_sum'] = yes_spec_power
    annot_yes['spec_pur'] = yes_spec_purs
    annot_yes['spec_cent'] = yes_spec_centroid
    annot_yes['spec_spread'] = yes_spec_spread
    annot_yes['spec_skew'] = yes_spec_skewness
    annot_yes['spec_kurt'] = yes_spec_kurtosis
    annot_yes['spec_slope'] = yes_spec_slope
    annot_yes['spec_roll'] = yes_spec_roll_off
    
    #create and fill dataframe for randomly selected noise slices
    annot_no = pd.DataFrame(columns = ['animal_number', 'session', 'time_stamp', 'Annotation'], index = np.arange(0,size_random))
    annot_no['animal_number'] = animal_number
    annot_no['session'] = session
    annot_no['Annotation'] = 'rand_noise'
    annot_no['time_stamp'] = data_no['slices'].values
    annot_no['power_sum'] = no_spec_power
    annot_no['spec_pur'] = no_spec_purs
    annot_no['spec_cent'] = no_spec_centroid
    annot_no['spec_spread'] = no_spec_spread
    annot_no['spec_skew'] = no_spec_skewness
    annot_no['spec_kurt'] = no_spec_kurtosis
    annot_no['spec_slope'] = no_spec_slope
    annot_no['spec_roll'] = no_spec_roll_off
    
    return annot_yes, annot_no 

Create data frame of annotation info

In [0]:
annot_path_cage = '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/annotation_data_frames/annot_df_homecage.csv'
annot_path_CPApost = '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/annotation_data_frames/annot_df_CPA.csv'
annot_path_pain = '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/annotation_data_frames/annot_df_pain.csv'

annot_paths = [annot_path_CPApost, annot_path_cage, annot_path_pain]

In [0]:
annot_data = pd.DataFrame()

for path in annot_paths:
    annot = pd.read_csv(path)
    annot = pd.DataFrame(data = annot)
    if 'radar' in annot['Annotation'].values:
      annot = annot[annot['Annotation'] != 'radar']
    if True in pd.isna(annot['Annotation']).values:
      annot['Annotation'] = ['BBC'] * annot['Annotation'].shape[0]
    
    print(annot.shape)
    print(annot.Annotation.value_counts())
    
    annot_data = annot_data.append(annot)

print(annot_data.shape)
annot_data.head()

(90, 6)
low slug      48
low multi     20
bbc           15
high slug      6
high multi     1
Name: Annotation, dtype: int64
(69, 6)
bbc           30
high slug     21
low slug      12
high multi     3
low multi      3
Name: Annotation, dtype: int64
(178, 6)
BBC    178
Name: Annotation, dtype: int64
(337, 6)


,Unnamed: 0,animal_number,session,time_stamp,Annotation,High Freq (Hz)
0,0,533,CPApair,46305.0,low slug,16500.0
2,2,533,CPApair,149692.5,low slug,16074.5
3,3,533,CPApair,243157.5,low slug,16500.0
4,4,533,CPApair,295560.0,low multi,27489.6
141,141,533,CPApair,376560.0,low slug,9940.8


Find path names for each netcdf file corresponding to wav file that has annotated data

In [0]:
netcdf_path_fear = '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcdf_files/Fear'

netcdf_path_pain = '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcdf_files/Pain'

In [0]:
path = netcdf_path_fear

path_names = []
files = os.listdir(path)
for file in files:
  path_names.append(path + "/" + file)

print(len(path_names))
path_names

16


['/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcdf_files/Fear/533_CPApair_xr_Dataset.nc',
 '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcdf_files/Fear/559_CPApair_xr_Dataset.nc',
 '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcdf_files/Fear/555_CPApair_xr_Dataset.nc',
 '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcdf_files/Fear/554_CPApair_xr_Dataset.nc',
 '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcdf_files/Fear/543_CPApair_xr_Dataset.nc',
 '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcdf_files/Fear/542_CPApair_xr_Dataset.nc',
 '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcdf_files/Fear/535_CPApair_xr_Dataset.nc',
 '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcdf_files/Fear/534_CPApair_xr_Dataset.nc',
 '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcdf_files/Fear/529_cagepair_xr_Dataset.nc',
 '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/netcd

In [0]:
order = 'animal'
size_random = 100
session_name = 'fear'
save_path = '/content/gdrive/Team Drives/USV_eScience_Incubator/Data/feature_data_frames'

annot_features_yes = pd.DataFrame()
annot_features_no = pd.DataFrame()

for path in path_names:
    
    annot_yes, annot_no = create_annotation_slice_features(path, order, annot_data, size_random)
    
    annot_features_yes = annot_features_yes.append(annot_yes, ignore_index=True)
    annot_features_no = annot_features_no.append(annot_no, ignore_index=True)

#create and save combined dataframe of yes and no
annot_features_yes.drop(['Unnamed: 0', 'High Freq (Hz)'], axis=1, inplace=True)
annot_features_full = pd.concat([annot_features_yes, annot_features_no])
print(annot_features_full.shape)
print(annot_features_full.Annotation.value_counts())

annot_features_full.to_csv(str(save_path + '/annot_8features_100noise_' + session_name + '_.csv'))

533 CPApair
533 CPApair
(26666,)
26666
559 CPApair
559 CPApair
(26666,)
26666
555 CPApair
555 CPApair
(26666,)
26666
554 CPApair
554 CPApair
(26666,)
26666
543 CPApair
543 CPApair
(26666,)
26666
542 CPApair
542 CPApair
(26666,)
26666
535 CPApair
535 CPApair
(26666,)
26666
534 CPApair
534 CPApair
(26666,)
26666
529 cagepair
529 cagepair
(26666,)
26666
527 cagepair
527 cagepair
(26666,)
26666
556 cagepair
556 cagepair
(26666,)
26666
553 cagepair
553 cagepair
(26666,)
26666
552 cagepair
552 cagepair
(26666,)
26666
541 cagepair
541 cagepair
(26666,)
26666
540 cagepair
540 cagepair
(26666,)
26666
557 cagepair
557 cagepair
(26666,)
26666
(1759, 12)
rand_noise    1600
low slug        60
bbc             45
high slug       27
low multi       23
high multi       4
Name: Annotation, dtype: int64
